# Desafio 6

Neste desafio, vamos praticar _feature engineering_, um dos processos mais importantes e trabalhosos de ML. Utilizaremos o _data set_ [Countries of the world](https://www.kaggle.com/fernandol/countries-of-the-world), que contém dados sobre os 227 países do mundo com informações sobre tamanho da população, área, imigração e setores de produção.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder, StandardScaler

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.datasets import load_digits, fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# Algumas configurações para o matplotlib.
%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [3]:
countries = pd.read_csv("countries.csv",decimal=',')

In [4]:
new_column_names = [
    "Country", "Region", "Population", "Area", "Pop_density", "Coastline_ratio",
    "Net_migration", "Infant_mortality", "GDP", "Literacy", "Phones_per_1000",
    "Arable", "Crops", "Other", "Climate", "Birthrate", "Deathrate", "Agriculture",
    "Industry", "Service"
]

countries.columns = new_column_names

countries.head(5)

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,48.0,0.00,23.06,163.07,700.0,36.0,3.2,12.13,0.22,87.65,1.0,46.60,20.34,0.380,0.240,0.380
1,Albania,EASTERN EUROPE,3581655,28748,124.6,1.26,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,74.49,3.0,15.11,5.22,0.232,0.188,0.579
2,Algeria,NORTHERN AFRICA,32930091,2381740,13.8,0.04,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,96.53,1.0,17.14,4.61,0.101,0.600,0.298
3,American Samoa,OCEANIA,57794,199,290.4,58.29,-20.71,9.27,8000.0,97.0,259.5,10.00,15.00,75.00,2.0,22.46,3.27,NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,152.1,0.00,6.60,4.05,19000.0,100.0,497.2,2.22,0.00,97.78,3.0,8.71,6.25,NaN,NaN,NaN


## Observações

Esse _data set_ ainda precisa de alguns ajustes iniciais. Primeiro, note que as variáveis numéricas estão usando vírgula como separador decimal e estão codificadas como strings. Corrija isso antes de continuar: transforme essas variáveis em numéricas adequadamente.

Além disso, as variáveis `Country` e `Region` possuem espaços a mais no começo e no final da string. Você pode utilizar o método `str.strip()` para remover esses espaços.

In [5]:
#chegando nessa parte do código, tive que colocar o paramentro decimal=',' durante a leitura do csv

In [6]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 20 columns):
Country             227 non-null object
Region              227 non-null object
Population          227 non-null int64
Area                227 non-null int64
Pop_density         227 non-null float64
Coastline_ratio     227 non-null float64
Net_migration       224 non-null float64
Infant_mortality    224 non-null float64
GDP                 226 non-null float64
Literacy            209 non-null float64
Phones_per_1000     223 non-null float64
Arable              225 non-null float64
Crops               225 non-null float64
Other               225 non-null float64
Climate             205 non-null float64
Birthrate           224 non-null float64
Deathrate           223 non-null float64
Agriculture         212 non-null float64
Industry            211 non-null float64
Service             212 non-null float64
dtypes: float64(16), int64(2), object(2)
memory usage: 35.5+ KB


In [7]:
#Remover espaços em branco das variáveis Country e Region
countries.Region = countries.Region.apply(lambda x : x.strip())
countries.Country = countries.Country.apply(lambda x : x.strip())

## Inicia sua análise a partir daqui

In [8]:
countries.shape

(227, 20)

In [9]:
countries.tail()

,Country,Region,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
222,West Bank,NEAR EAST,2460492,5860,419.9,0.00,2.98,19.62,800.0,NaN,145.2,16.90,18.97,64.13,3.0,31.67,3.92,0.090,0.280,0.630
223,Western Sahara,NORTHERN AFRICA,273008,266000,1.0,0.42,NaN,NaN,NaN,NaN,NaN,0.02,0.00,99.98,1.0,NaN,NaN,NaN,NaN,0.400
224,Yemen,NEAR EAST,21456188,527970,40.6,0.36,0.00,61.50,800.0,50.2,37.2,2.78,0.24,96.98,1.0,42.89,8.30,0.135,0.472,0.393
225,Zambia,SUB-SAHARAN AFRICA,11502010,752614,15.3,0.00,0.00,88.29,800.0,80.6,8.2,7.08,0.03,92.90,2.0,41.00,19.93,0.220,0.290,0.489
226,Zimbabwe,SUB-SAHARAN AFRICA,12236805,390580,31.3,0.00,0.00,67.69,1900.0,90.7,26.8,8.32,0.34,91.34,2.0,28.01,21.84,0.179,0.243,0.579


In [10]:
countries.describe()

,Population,Area,Pop_density,Coastline_ratio,Net_migration,Infant_mortality,GDP,Literacy,Phones_per_1000,Arable,Crops,Other,Climate,Birthrate,Deathrate,Agriculture,Industry,Service
count,2.270000e+02,2.270000e+02,227.000000,227.000000,224.000000,224.000000,226.000000,209.000000,223.000000,225.000000,225.000000,225.000000,205.000000,224.000000,223.000000,212.000000,211.000000,212.000000
mean,2.874028e+07,5.982270e+05,379.047137,21.165330,0.038125,35.506964,9689.823009,82.838278,236.061435,13.797111,4.564222,81.638311,2.139024,22.114732,9.241345,0.150844,0.282711,0.565283
std,1.178913e+08,1.790282e+06,1660.185825,72.286863,4.889269,35.389899,10049.138513,19.722173,227.991829,13.040402,8.361470,16.140835,0.699397,11.176716,4.990026,0.146798,0.138272,0.165841
min,7.026000e+03,2.000000e+00,0.000000,0.000000,-20.990000,2.290000,500.000000,17.600000,0.200000,0.000000,0.000000,33.330000,1.000000,7.290000,2.290000,0.000000,0.020000,0.062000
25%,4.376240e+05,4.647500e+03,29.150000,0.100000,-0.927500,8.150000,1900.000000,70.600000,37.800000,3.220000,0.190000,71.650000,2.000000,12.672500,5.910000,0.037750,0.193000,0.429250
50%,4.786994e+06,8.660000e+04,78.800000,0.730000,0.000000,21.000000,5550.000000,92.500000,176.200000,10.420000,1.030000,85.700000,2.000000,18.790000,7.840000,0.099000,0.272000,0.571000
75%,1.749777e+07,4.418110e+05,190.150000,10.345000,0.997500,55.705000,15700.000000,98.000000,389.650000,20.000000,4.440000,95.440000,3.000000,29.820000,10.605000,0.221000,0.341000,0.678500
max,1.313974e+09,1.707520e+07,16271.500000,870.660000,23.060000,191.190000,55100.000000,100.000000,1035.600000,62.110000,50.680000,100.000000,4.000000,50.730000,29.740000,0.769000,0.906000,0.954000


In [11]:
countries.isnull().sum()

Country              0
Region               0
Population           0
Area                 0
Pop_density          0
Coastline_ratio      0
Net_migration        3
Infant_mortality     3
GDP                  1
Literacy            18
Phones_per_1000      4
Arable               2
Crops                2
Other                2
Climate             22
Birthrate            3
Deathrate            4
Agriculture         15
Industry            16
Service             15
dtype: int64

## Questão 1

Quais são as regiões (variável `Region`) presentes no _data set_? Retorne uma lista com as regiões únicas do _data set_ com os espaços à frente e atrás da string removidos (mas mantenha pontuação: ponto, hífen etc) e ordenadas em ordem alfabética.

In [16]:
countries['Region'].unique().tolist()

['ASIA (EX. NEAR EAST)',
 'EASTERN EUROPE',
 'NORTHERN AFRICA',
 'OCEANIA',
 'WESTERN EUROPE',
 'SUB-SAHARAN AFRICA',
 'LATIN AMER. & CARIB',
 'C.W. OF IND. STATES',
 'NEAR EAST',
 'NORTHERN AMERICA',
 'BALTICS']

In [17]:
def q1():
    return countries.Region.sort_values().unique().tolist()

In [18]:
#countries.Region.sort_values().unique().tolist()
#sorted(countries['Region'].unique())
#list(countries.Region.sort_values().unique())

## Questão 2

Discretizando a variável `Pop_density` em 10 intervalos com `KBinsDiscretizer`, seguindo o encode `ordinal` e estratégia `quantile`, quantos países se encontram acima do 90º percentil? Responda como um único escalar inteiro.

In [19]:
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html
est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
estX = est.fit_transform(countries[['Pop_density']])
estX[:,0] , sum(estX[:,0]==9)
#O estX retorna os bins.

(array([3., 6., 1., 8., 7., 0., 6., 7., 1., 5., 8., 0., 5., 5., 2., 9., 9.,
        9., 3., 8., 1., 4., 9., 3., 0., 5., 0., 2., 7., 4., 4., 3., 4., 8.,
        4., 2., 0., 5., 7., 0., 0., 2., 6., 2., 8., 2., 1., 5., 5., 3., 5.,
        5., 5., 6., 6., 1., 5., 7., 4., 3., 5., 8., 1., 3., 2., 4., 2., 3.,
        1., 6., 0., 4., 0., 6., 9., 4., 7., 5., 9., 5., 0., 8., 8., 8., 6.,
        9., 2., 3., 0., 8., 4., 9., 6., 0., 8., 6., 3., 4., 3., 6., 8., 7.,
        7., 8., 9., 4., 0., 3., 6., 7., 9., 6., 2., 2., 2., 8., 4., 2., 0.,
        7., 3., 7., 9., 5., 2., 6., 4., 9., 0., 9., 0., 8., 0., 9., 9., 3.,
        7., 6., 9., 0., 5., 4., 2., 0., 9., 7., 9., 7., 1., 1., 3., 0., 6.,
        7., 1., 1., 7., 3., 3., 1., 1., 2., 8., 6., 6., 8., 4., 8., 5., 0.,
        8., 1., 6., 8., 2., 8., 4., 9., 7., 1., 4., 6., 7., 5., 9., 6., 5.,
        1., 1., 2., 5., 8., 1., 0., 4., 1., 7., 5., 9., 3., 3., 6., 5., 7.,
        7., 4., 5., 1., 3., 9., 6., 4., 2., 7., 2., 1., 4., 1., 2., 8., 3.,
        3., 

In [20]:
def q2():
    est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
    estX = est.fit_transform(countries[['Pop_density']])
    return sum(estX[:,0]==9)

# Questão 3

Se codificarmos as variáveis `Region` e `Climate` usando _one-hot encoding_, quantos novos atributos seriam criados? Responda como um único escalar.

def q3():
    features_encoded = pd.get_dummies(countries[['Region', 'Climate']].fillna(''))
    return int(features_encoded.shape[1])

In [21]:
encoder = OneHotEncoder(sparse = False).fit_transform(countries[['Region', 'Climate']].fillna(0))   
int(encoder.shape[1])

18

In [23]:
encoded = pd.get_dummies(countries[['Region', 'Climate']].fillna(''))
int(encoded.shape[1])

18

In [25]:
def q3():
    encoder = OneHotEncoder(sparse = False).fit_transform(countries[['Region', 'Climate']].fillna(0))   
    return int(encoder.shape[1])

## Questão 4

Aplique o seguinte _pipeline_:

1. Preencha as variáveis do tipo `int64` e `float64` com suas respectivas medianas.
2. Padronize essas variáveis.

Após aplicado o _pipeline_ descrito acima aos dados (somente nas variáveis dos tipos especificados), aplique o mesmo _pipeline_ (ou `ColumnTransformer`) ao dado abaixo. Qual o valor da variável `Arable` após o _pipeline_? Responda como um único float arredondado para três casas decimais.

In [26]:
test_country = [
    'Test Country', 'NEAR EAST', -0.19032480757326514,
    -0.3232636124824411, -0.04421734470810142, -0.27528113360605316,
    0.13255850810281325, -0.8054845935643491, 1.0119784924248225,
    0.6189182532646624, 1.0074863283776458, 0.20239896852403538,
    -0.043678728558593366, -0.13929748680369286, 1.3163604645710438,
    -0.3699637766938669, -0.6149300604558857, -0.854369594993175,
    0.263445277972641, 0.5712416961268142
]

In [27]:
#Criando o pipeline pra inputar a mediana e padronizar
num_pipeline = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")), 
        ('scale', StandardScaler())])

In [28]:
#criando uma lista com as variáveis numéricas
col_num = countries.describe().columns.tolist()

#Aplicando o fit_transform nos dados do dataset
pipeline_transformation = num_pipeline.fit_transform(countries[col_num])

#aplicando o fit no dado novo
test_country_transform = num_pipeline.transform([test_country[2:]])

In [29]:
#index da coluna countries.columns.get_loc("Arable")
round(test_country_transform[:,countries.columns.get_loc("Arable")-2][0],3)

-1.047

In [30]:
def q4():
    num_pipeline = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")), 
        ('scale', StandardScaler())])
    
    col_num = countries.describe().columns.tolist()

    pipeline_transformation = num_pipeline.fit_transform(countries[col_num])
    test_country_transform = num_pipeline.transform([test_country[2:]])
    
    return round(test_country_transform[:,countries.columns.get_loc("Arable")-2][0],3)

## Questão 5

Descubra o número de _outliers_ da variável `Net_migration` segundo o método do _boxplot_, ou seja, usando a lógica:

$$x \notin [Q1 - 1.5 \times \text{IQR}, Q3 + 1.5 \times \text{IQR}] \Rightarrow x \text{ é outlier}$$

que se encontram no grupo inferior e no grupo superior.

Você deveria remover da análise as observações consideradas _outliers_ segundo esse método? Responda como uma tupla de três elementos `(outliers_abaixo, outliers_acima, removeria?)` ((int, int, bool)).

In [31]:
Q1 = countries['Net_migration'].quantile(0.25)
Q3 = countries['Net_migration'].quantile(0.75)
#faixa entre os qrs
iqr = Q3 - Q1

In [32]:
#Tudo que estiver fora da faixa Q1 - 1.5 *IQR, Q3 + 1.5 * IQR é considerado outlier
non_outlier_interval_iqr = [Q1 - 1.5 * iqr, Q3 + 1.5 * iqr]

print(f"Faixa considerada \"normal\": {non_outlier_interval_iqr}")

outliers_abaixo = len(countries['Net_migration'][(countries['Net_migration'] < non_outlier_interval_iqr[0])])
outliers_acima= len(countries['Net_migration'][(countries['Net_migration'] > non_outlier_interval_iqr[1])])

Faixa considerada "normal": [-3.8149999999999995, 3.885]


In [33]:
def q5():
    q1 = countries['Net_migration'].quantile(0.25)
    q3 = countries['Net_migration'].quantile(0.75)
    iqr = q3 - q1

    non_outlier_interval_iqr = [q1 - 1.5 * iqr, q3 + 1.5 * iqr]

    print(f"Faixa considerada \"normal\": {non_outlier_interval_iqr}")

    outliers_abaixo = len(countries['Net_migration'][(countries['Net_migration'] < non_outlier_interval_iqr[0])])
    outliers_acima= len(countries['Net_migration'][(countries['Net_migration'] > non_outlier_interval_iqr[1])])
    
    return tuple([outliers_abaixo,outliers_acima,False])

## Questão 6
Para as questões 6 e 7 utilize a biblioteca `fetch_20newsgroups` de datasets de test do `sklearn`

Considere carregar as seguintes categorias e o dataset `newsgroups`:

```
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)
```


Aplique `CountVectorizer` ao _data set_ `newsgroups` e descubra o número de vezes que a palavra _phone_ aparece no corpus. Responda como um único escalar.

#### Sobre CountVectorizer
    Ao construir modelos para análise de texto não podemos trabalhar diretamente com os dados na forma textual, pois o computador trabalha com dados na forma numérica. Para analisar dados de texto é preciso separar o texto em palavras ou em conjunto de palavras, através do processo de tokenization, e transformar essas palavras em representação numérica, através do processo de extração de características (features).
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 

    O conceito deste método é contar as vezes em que a palavra aparece no documento e usa esse valor como atributo (maior frequência = maior peso). 
    O problema dessa abordagem é que, a frequência que uma palavra aparece não necessariamente a torna mais relevante. Caso não seja feito um pré-processamento do texto, palavras que não carregam informações relevantes como ‘a’,’the’,’an’,’o’ aparecerão com mais frequência em relação a palavras com informações relevantes, prejudicando o modelo.

In [34]:
categories = ['sci.electronics', 'comp.graphics', 'rec.motorcycles']
newsgroup = fetch_20newsgroups(subset="train", categories=categories, shuffle=True, random_state=42)

In [35]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
count_vectorizer = CountVectorizer()
cv = count_vectorizer.fit_transform(newsgroup.data)

In [36]:
#Representação da palavra phone
count_vectorizer.vocabulary_['phone']

19211

In [37]:
cv[:,count_vectorizer.vocabulary_['phone']]

<1773x1 sparse matrix of type '<class 'numpy.int64'>'
	with 143 stored elements in Compressed Sparse Row format>

In [38]:
def q6():
    count_vectorizer = CountVectorizer()
    cv = count_vectorizer.fit_transform(newsgroup.data)
    return cv[:,count_vectorizer.vocabulary_['phone']].sum()

## Questão 7

Aplique `TfidfVectorizer` ao _data set_ `newsgroups` e descubra o TF-IDF da palavra _phone_. Responda como um único escalar arredondado para três casas decimais.

#### Sobre Tfidf
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer
    O conceito desse método é dar destaque para as palavras que não aparecem com tanta frequência nos documentos. Essa abordagem supre o problema do CountVectorizer; palavras com alta frequência como "a" e "the" terão pontuações muito baixas (pois serão exibidas em muitos documentos), enquanto palavras com pouca frequência terão pontuações mais altas e, portanto, elas serão as que o modelo identificará como importantes e tentará aprender.
    
       Para o cálculo, o método utiliza da sequinte equação: 
       * TF= (frequencia da palavra no documento/quantidade de palavras no documento) 
       * IDF= log(quantidade de documentos/ quantidade de documentos onde a palavra está presente)
       * TFIDF= TF * IDF

In [39]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidf = TfidfVectorizer()
tfidf_ = tfidf.fit_transform(newsgroup.data)

In [40]:
tfidf.vocabulary_['phone']

19211

In [41]:
round(tfidf_[:,count_vectorizer.vocabulary_['phone']].sum(),3)

8.888

In [42]:
def q7():
    tfidf = TfidfVectorizer()
    tfidf_ = tfidf.fit_transform(newsgroup.data)
    
    return round(tfidf_[:,count_vectorizer.vocabulary_['phone']].sum(),3)